<a href="https://colab.research.google.com/github/Trodriguez238/PROGCOM-B/blob/main/GAQ2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox, scrolledtext
import google.generativeai as genai

# --- Configuración inicial ---
API_KEY = ""  # Coloca tu API Key aquí
model = None
numero_secreto = ""
intentos = []

# --- Validación ---
def validar_numero(numero):
    return len(numero) == 4 and numero.isdigit() and len(set(numero)) == 4

# --- Configurar Gemini ---
def configurar_gemini():
    global model
    try:
        genai.configure(api_key=API_KEY)
        model = genai.GenerativeModel('gemini-2.5-flash')
        return True
    except Exception as e:
        messagebox.showerror("Error", f"No se pudo configurar Gemini:\n{e}")
        return False

# --- Generar número secreto ---
def generar_numero():
    prompt = "Genera un número de 4 dígitos diferentes. Solo responde con el número, sin texto."
    try:
        response = model.generate_content(prompt)
        numero = response.text.strip()
        if validar_numero(numero):
            return numero
    except:
        pass
    return None

# --- Evaluar intento con explicación ---
def evaluar_con_explicacion(intento):
    prompt = f"""
    Estás jugando Picas y Fijas.
    Número secreto: {numero_secreto}
    Intento del jugador: {intento}

    Evalúa el intento y explica el resultado:
    - Indica cuántas fijas y picas hay.
    - Explica por qué cada dígito es pica o fija.
    - Da una sugerencia para el siguiente intento.

    Formato:
    Fijas: X
    Picas: Y
    Explicación: ...
    Sugerencia: ...
    """
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Error al generar explicación: {e}"

# --- Interfaz gráfica ---
def iniciar_juego():
    global numero_secreto, intentos
    if not configurar_gemini():
        return
    numero_secreto = generar_numero()
    if not numero_secreto:
        messagebox.showerror("Error", "Gemini no generó un número válido.")
        return
    intentos.clear()
    entrada.config(state="normal")
    boton_intentar.config(state="normal")
    texto_estado.config(text="Número secreto generado. ¡Empieza a jugar!")

def realizar_intento():
    intento = entrada.get().strip()
    if not validar_numero(intento):
        messagebox.showwarning("Error", "Ingresa 4 dígitos diferentes.")
        return

    resultado = evaluar_con_explicacion(intento)
    intentos.append((intento, resultado))
    actualizar_historial()
    entrada.delete(0, tk.END)

    if "Fijas: 4" in resultado:
        texto_estado.config(text=f"¡Ganaste en {len(intentos)} intentos!")
        entrada.config(state="disabled")
        boton_intentar.config(state="disabled")

def actualizar_historial():
    historial_texto.config(state="normal")
    historial_texto.delete(1.0, tk.END)
    for i, (num, res) in enumerate(intentos, 1):
        historial_texto.insert(tk.END, f"#{i} → {num}\n{res}\n\n")
    historial_texto.config(state="disabled")

# --- GUI ---
root = tk.Tk()
root.title("Picas y Fijas Educativo con Gemini")
root.geometry("600x700")
root.configure(bg="#2d3748")

tk.Label(root, text="Picas y Fijas con Explicación", font=("Arial", 20), bg="#2d3748", fg="white").pack(pady=10)

entrada = tk.Entry(root, font=("Arial", 20), width=10, justify="center")
entrada.pack(pady=10)
entrada.bind("<Return>", lambda e: realizar_intento())

boton_intentar = tk.Button(root, text="Intentar", font=("Arial", 12), command=realizar_intento)
boton_intentar.pack(pady=5)

boton_iniciar = tk.Button(root, text="Iniciar Juego", font=("Arial", 12), command=iniciar_juego)
boton_iniciar.pack(pady=5)

texto_estado = tk.Label(root, text="Configura la API y comienza", font=("Arial", 10), bg="#2d3748", fg="#fbd38d")
texto_estado.pack(pady=10)

historial_texto = scrolledtext.ScrolledText(root, font=("Courier", 12), height=20, bg="#1a202c", fg="white")
historial_texto.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
historial_texto.config(state="disabled")

root.mainloop()
